In [3]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import tweepy
import json
from timeit import default_timer as timer
from tweepy import OAuthHandler

In [7]:
# Loading dataset

url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"

response = requests.get(url)


In [14]:
with open('image_predictions.tsv', mode = 'wb') as file:
          file.write(response.content)

In [20]:
df_image = pd.read_csv(r"C:\Users\sibin\Desktop\udacity\We_rate_dogs_tweepy\data\image_predictions.tsv", sep = '\t')

In [21]:
df_image.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [51]:
#load twitter archieve data
df_twitter = pd.read_csv(r"C:\Users\sibin\Desktop\udacity\We_rate_dogs_tweepy\data\twitter-archive-enhanced.csv")
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [27]:
# importing twitter api
import tweepy
from tweepy import OAuthHandler
import json
from timeit import default_timer as timer


consumer_key ='TZ24AcbFwrJZtqtXD5YY0gyzk'
consumer_secret ='27Y8KvnjW6Uq6Frldp9mM3Of928ehqi5ApVZHrPKD9QPbNM17s'
access_token ='2923733863-ToNaf6imyfeK4R2uekG1QTtT23x0QGGPWDb4pcl'
access_secret = 'czPyNcFFDdc51X8Hu59oT1WybjtXcba2OIFDzrJC4pLhG'

In [28]:

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

In [46]:
tweet_ids = df_twitter.tweet_id.values
with open('tweet_json.txt', 'w') as output:
    for tweet_id in tweet_ids:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, output)
            output.write('\n')
        except tweepy.TweepError:
            print('Error: tweet missing')

Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing
Error: tweet missing


In [48]:

# Converting relevant JSON data to dataframe

tweets_list =[]

with open('tweet_json.txt') as json_file:
    for line in json_file:
    
        tweets_dict = {}
        tweets_json = json.loads(line)
        
        try:
            tweets_dict['tweet_id'] = tweets_json['extended_entities']['media'][0]['id']
        except:
            tweets_dict['tweet_id'] = 'na'

        tweets_dict['retweet_count'] = tweets_json['retweet_count']
        tweets_dict['favorite_count'] = tweets_json['favorite_count']
        
        tweets_list.append(tweets_dict)

In [131]:
df_tweets_json = pd.DataFrame(tweets_list)


In [132]:
# For loop which will add each available tweet to a new line of tweet_json.txt
with open('tweets_json.txt', 'a', encoding='utf8') as f:
    for tweet_id in df_twitter['tweet_id']:
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            json.dump(tweet._json, f)
            f.write('\n')
        except:
            continue

In [135]:
# For loop to append each tweet into a list
tweets_data = []

tweet_file = open('tweets_json.txt', "r")

for line in tweet_file:
    try:
        tweet = json.loads(line)
        tweets_data.append(tweet)
    except:
        continue
        
tweet_file.close()

In [153]:
# Create tweet_info DataFrame
df_info_json = pd.DataFrame()

In [154]:
 #Add variables to tweet_info_json DataFrame
df_info_json['tweet_id'] = list(map(lambda tweet: tweet['id'], tweets_data))
df_info_json['retweet_count'] = list(map(lambda tweet: tweet['retweet_count'], tweets_data))
df_info_json['favorite_count'] = list(map(lambda tweet: tweet['favorite_count'], tweets_data))

### Assessing data

#### 1. Twitter Archieve data

In [97]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

    There are 2356 entries.
    retweeted_status_id shows the tweets that are retweeted.
    The dog stages are given as four columns
    There are 78 reply tweets
    181 are retweets(retweeted_status_id)
    2297 rows have expanded url, which are the links to tweets, 59 image url missing

In [99]:
df_twitter.head(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [101]:
df_twitter['source'].unique()

array(['<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
       '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
       '<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>',
       '<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>'],
      dtype=object)

    There are 4 sources from where data is extracted, we dont need the full url. 

In [114]:
# timestamp is an object
type(df_twitter['timestamp'][0])

str

    Time stamp is in string format

In [72]:
# number of missing values
df_twitter.isnull().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [199]:
# said that denominator needs to be 10
df_twitter[df_twitter['rating_denominator'] != 10]['rating_denominator'].value_counts()

50     3
11     3
20     2
80     2
90     1
40     1
130    1
2      1
70     1
7      1
110    1
170    1
120    1
15     1
16     1
150    1
0      1
Name: rating_denominator, dtype: int64

    There are 23 denominators with value not equal to 10 

In [202]:
# numerators are said to have a series of values
df_twitter['rating_numerator'].value_counts().sort_values(ascending = False)

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
121       1
80        1
204       1
24        1
143       1
99        1
45        1
27        1
17        1
1776      1
960       1
666       1
182       1
26        1
144       1
88        1
84        1
165       1
60        1
50        1
44        1
20        1
Name: rating_numerator, dtype: int64

In [208]:
len(df_twitter[df_twitter['rating_numerator'] > 20]['rating_numerator'])

24

In [452]:
# View rows where the value of 'name' is lowercase - indicates that it is not an actual name
x = df_twitter.loc[df_twitter['name'].str.islower()]
x.name

22       such
56          a
118     quite
169     quite
193     quite
        ...  
2349       an
2350        a
2352        a
2353        a
2354        a
Name: name, Length: 109, dtype: object

    There are 109 names of lowercase that are not actually names

In [106]:
# check for duplicated values
df_twitter.duplicated().sum()

0

    There are zero duplicated rows

In [ ]:
df_twitter['']

#### 2. Twitter image data

In [94]:
df_image.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [96]:
df_image.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


    data seems to have no missing values. 

#### 3. Tweet json file

In [166]:
df_info_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   tweet_id        2331 non-null   int64
 1   retweet_count   2331 non-null   int64
 2   favorite_count  2331 non-null   int64
dtypes: int64(3)
memory usage: 54.8 KB


    There are 2331 tweets with zero null values,
    It would be good if we could merge this data to twitter archieve data

In [167]:
# missing values
df_info_json.isnull().sum()

tweet_id          0
retweet_count     0
favorite_count    0
dtype: int64

    There are zero missing values

In [170]:
df_info_json[df_info_json.tweet_id.duplicated()]

,tweet_id,retweet_count,favorite_count


    There are zero duplicated values

### Assessment Summary

#### Quality
######  `df_twitter` table

- 181 Retweeted data
- 78 reply tweets
- Source column with HTML tags
- Expanded URLS have 59 missing values
- Timestamp in string format
- 23 denominoators with value not equal to 10
- 24 tweets with numerator more than 20, need to see how many of them are retweets.
- Multiple dog stages need to combined to one

###### `df_image` table

- Only 2075 tweets have image

#### Tidiness
- In df_twitter table dog's stage dogoo, floofer, pupper, puppo is spread in different columns
- df_image , df_info_json needs to be joined with archive table df_twitter
- columns associated with retweets and reply tweets




## Cleaning Data

In [456]:
# create copies of the dataframes for cleaning
df_twitter_clean = df_twitter.copy()
df_image_clean = df_image.copy()
df_info_json_clean = df_info_json.copy()

##### 1. 181 retweeted data
##### Define:
   There are 181 retweets, we need only the original data. So Drop all rows containing non null values for retweeted_status_id, retweeted_status_user_id and retweeted_status_timestamp.

##### Code

In [457]:
df_twitter_clean = df_twitter_clean[df_twitter_clean['retweeted_status_id'].isna()]

##### Test

In [458]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2175 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2175 non-null   object 
 4   source                      2175 non-null   object 
 5   text                        2175 non-null   object 
 6   retweeted_status_id         0 non-null      float64
 7   retweeted_status_user_id    0 non-null      float64
 8   retweeted_status_timestamp  0 non-null      object 
 9   expanded_urls               2117 non-null   object 
 10  rating_numerator            2175 non-null   int64  
 11  rating_denominator          2175 non-null   int64  
 12  name                        2175 non-null   object 
 13  doggo                       2175 

##### 78 reply tweets
##### Define
2 . inreply_status_id shows the reply tweets, they are to be removed

##### Code

In [459]:
df_twitter_clean = df_twitter_clean[df_twitter_clean['in_reply_to_status_id'].isna()]

##### Test

In [460]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2097 non-null   int64  
 1   in_reply_to_status_id       0 non-null      float64
 2   in_reply_to_user_id         0 non-null      float64
 3   timestamp                   2097 non-null   object 
 4   source                      2097 non-null   object 
 5   text                        2097 non-null   object 
 6   retweeted_status_id         0 non-null      float64
 7   retweeted_status_user_id    0 non-null      float64
 8   retweeted_status_timestamp  0 non-null      object 
 9   expanded_urls               2094 non-null   object 
 10  rating_numerator            2097 non-null   int64  
 11  rating_denominator          2097 non-null   int64  
 12  name                        2097 non-null   object 
 13  doggo                       2097 

##### 3 . Multiple dog stages need to combined to one

##### Define:
There are columns with multiple dog stages, create one column with the dog stage

##### Code:

In [461]:
df_twitter['floofer'].replace(np.nan , "", inplace = True)
df_twitter['pupper'].replace( np.nan, "", inplace = True)
df_twitter['puppo'].replace(np.nan , "", inplace = True)
df_twitter['doggo'].replace(np.nan , "", inplace = True)
df_twitter['floofer'].replace("None" , "", inplace = True)
df_twitter['pupper'].replace("None" , "", inplace = True)
df_twitter['puppo'].replace("None" , "", inplace = True)
df_twitter['doggo'].replace("None" , "", inplace = True)

In [462]:
df_twitter_clean['stage'] = df_twitter_clean.doggo + df_twitter_clean.floofer + df_twitter_clean.pupper + df_twitter_clean.puppo
df_twitter_clean.loc[df_twitter_clean.stage == 'doggopupper', 'stage'] = 'doggo, pupper'
df_twitter_clean.loc[df_twitter_clean.stage == 'doggopuppo', 'stage'] = 'doggo, puppo'
df_twitter_clean.loc[df_twitter_clean.stage == 'doggofloofer', 'stage'] = 'doggo, floofer'

##### Test: 

In [463]:
df_twitter_clean.stage.value_counts()

                  1761
pupper             221
doggo               72
puppo               23
floofer              9
doggo, pupper        9
doggo, floofer       1
doggo, puppo         1
Name: stage, dtype: int64

    Here space is treated as a value, it should be null value

In [464]:
df_twitter_clean.replace("",np.nan,inplace = True)

In [465]:
#checking again
df_twitter_clean.stage.value_counts()

pupper            221
doggo              72
puppo              23
floofer             9
doggo, pupper       9
doggo, floofer      1
doggo, puppo        1
Name: stage, dtype: int64

#### Tidiness
1 . In df_tweets table dog's stage dogoo, floofer, pupper, puppo is spread in different columns

##### Define: 
As we have combined the stage values to a single column, we need to drop the individual stage columns


##### Code

In [466]:
col = ['doggo','puppo','floofer','pupper']
df_twitter_clean.drop(col, axis = 1, inplace = True)

##### Test

In [467]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2097 non-null   int64  
 1   in_reply_to_status_id       0 non-null      float64
 2   in_reply_to_user_id         0 non-null      float64
 3   timestamp                   2097 non-null   object 
 4   source                      2097 non-null   object 
 5   text                        2097 non-null   object 
 6   retweeted_status_id         0 non-null      float64
 7   retweeted_status_user_id    0 non-null      float64
 8   retweeted_status_timestamp  0 non-null      float64
 9   expanded_urls               2094 non-null   object 
 10  rating_numerator            2097 non-null   int64  
 11  rating_denominator          2097 non-null   int64  
 12  name                        2097 non-null   object 
 13  stage                       336 n

##### Tidiness
##### 2 . columns associated with retweets and reply tweets
##### Define
Columns associated with retweets(retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp ) and reply tweets(in_reply_to_status_id,in_reply_to_user_id ) needs to be droped

##### Code 

In [468]:
col = ['retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','in_reply_to_status_id','in_reply_to_user_id']
df_twitter_clean.drop(col, axis = 1, inplace = True)

##### Test

In [469]:
df_twitter_clean.head(3)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,stage
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,NaN
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,NaN
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,NaN


##### 4. Expanded Urls with missing values

    We only want thhe tweets that have an image, expanded urls are the links to tweets, and they are said to have an image associated with them. Whereas in dataframe df_image, we have downloaded the image urls of the tweets.


In [470]:
sum(df_twitter_clean.expanded_urls.isnull())

3

    We have 3 tweets with urls missing, we need to see if dataframe df_image have the urls for these tweets, if they do then we will keep this tweets in table or else remove these tweets

In [471]:
# generating the tweets with no expanded urls
df_twitter_x = df_twitter_clean[df_twitter_clean['expanded_urls'].isna()]

In [472]:
df_twitter_x['tweet_id'].values

array([828361771580813312, 785515384317313025, 696518437233913856],
      dtype=int64)

In [473]:
# we need to see if df_image_clean have the image urls for this three tweets
id = [828361771580813312,85515384317313025 ,696518437233913856]

for i in id:
    df_image_x = df_image_clean[df_image_clean['tweet_id'] == i]

In [474]:
df_image_x

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


    It is seen that df_image doesnt have the urls associated with the three tweets.

##### Define

Remove the rows with expanded urls missing

##### Code:

In [475]:
df_twitter_clean.dropna(subset = ['expanded_urls'], inplace = True)

##### Test

In [476]:
sum(df_twitter_clean.expanded_urls.isnull())

0

In [477]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2094 entries, 0 to 2355
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_id            2094 non-null   int64 
 1   timestamp           2094 non-null   object
 2   source              2094 non-null   object
 3   text                2094 non-null   object
 4   expanded_urls       2094 non-null   object
 5   rating_numerator    2094 non-null   int64 
 6   rating_denominator  2094 non-null   int64 
 7   name                2094 non-null   object
 8   stage               335 non-null    object
dtypes: int64(3), object(6)
memory usage: 163.6+ KB


#### 5. Timestamp in string format

##### Define:
Timestamp needs to be converted to DateTime

##### Code

In [478]:
df_twitter_clean['timestamp'] = pd.to_datetime(df_twitter_clean['timestamp'], format='%Y-%m-%d %H:%M:%S')


##### Test

In [479]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2094 entries, 0 to 2355
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            2094 non-null   int64              
 1   timestamp           2094 non-null   datetime64[ns, UTC]
 2   source              2094 non-null   object             
 3   text                2094 non-null   object             
 4   expanded_urls       2094 non-null   object             
 5   rating_numerator    2094 non-null   int64              
 6   rating_denominator  2094 non-null   int64              
 7   name                2094 non-null   object             
 8   stage               335 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(3), object(5)
memory usage: 163.6+ KB


#### 6. Source column with HTML tags. 

##### Define
There are major four sources from where data is extracted, but it is provided in HTML format, we need to show the sources as such.    
Extract the sources from HTML tags

##### Code

In [480]:
df_twitter_clean['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     1962
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       30
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [481]:
df_twitter_clean['source'] = df_twitter_clean['source'].str.extract('^<a.+>(.+)</a>$')


##### Test

In [482]:
df_twitter_clean['source'].value_counts()

Twitter for iPhone     1962
Vine - Make a Scene      91
Twitter Web Client       30
TweetDeck                11
Name: source, dtype: int64

#### 7. 23 tweets with denominoators value not equal to 10

##### Define:
    We only want denominators with value 10, drop the ones with denominator not equal 10

##### Code:

In [483]:
df_twitter_clean = df_twitter_clean[df_twitter_clean['rating_denominator'] == 10]

##### Test

In [484]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2077 entries, 0 to 2355
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            2077 non-null   int64              
 1   timestamp           2077 non-null   datetime64[ns, UTC]
 2   source              2077 non-null   object             
 3   text                2077 non-null   object             
 4   expanded_urls       2077 non-null   object             
 5   rating_numerator    2077 non-null   int64              
 6   rating_denominator  2077 non-null   int64              
 7   name                2077 non-null   object             
 8   stage               335 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(3), object(5)
memory usage: 162.3+ KB


In [485]:
df_twitter_clean.rating_denominator.value_counts()

10    2077
Name: rating_denominator, dtype: int64

##### 8 .  tweets with numerator more than 20

##### Define: 
Delete the rows with numerator greater than 20.

##### Code:

In [486]:
# checking the rows with numerator greater than 20
df_twitter_clean_x = df_twitter_clean[df_twitter_clean['rating_numerator'] >20 ]


In [487]:
df_twitter_clean_x.rating_numerator	.value_counts()

27      1
420     1
75      1
26      1
1776    1
Name: rating_numerator, dtype: int64

In [488]:
df_twitter_clean = df_twitter_clean[df_twitter_clean['rating_numerator'] <=20 ]


In [489]:
df_twitter_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2072 entries, 0 to 2355
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            2072 non-null   int64              
 1   timestamp           2072 non-null   datetime64[ns, UTC]
 2   source              2072 non-null   object             
 3   text                2072 non-null   object             
 4   expanded_urls       2072 non-null   object             
 5   rating_numerator    2072 non-null   int64              
 6   rating_denominator  2072 non-null   int64              
 7   name                2072 non-null   object             
 8   stage               334 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(3), object(5)
memory usage: 161.9+ KB


    We have the five rows removed.

#### 9 .Resolving the name issue

##### Define:
None and names with lower character needs to be converted to nan.

In [512]:
df_twitter_clean['name'].value_counts()

None       587
a           54
Charlie     11
Lucy        11
Cooper      10
          ... 
Reptar       1
Jersey       1
Hamrick      1
Steve        1
Creg         1
Name: name, Length: 951, dtype: int64

In [522]:
# generating names starting with lowercases
x = df_twitter_clean.loc[df_twitter_clean['name'].str.islower()]

In [526]:
y =x.name.unique()

In [527]:
# reolace lowercase names with nan
df_twitter_clean['name'] = df_twitter_clean['name'].replace(y, np.nan)

In [528]:
# replace none with nan
df_twitter_clean['name'] = df_twitter_clean['name'].replace('None', np.nan)

##### Test

In [552]:
df_twitter_clean.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2072 entries, 0 to 2355
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            2072 non-null   int64              
 1   timestamp           2072 non-null   datetime64[ns, UTC]
 2   source              2072 non-null   object             
 3   text                2072 non-null   object             
 4   expanded_urls       2072 non-null   object             
 5   rating_numerator    2072 non-null   int64              
 6   rating_denominator  2072 non-null   int64              
 7   name                1384 non-null   object             
 8   stage               334 non-null    object             
dtypes: datetime64[ns, UTC](1), int64(3), object(5)
memory usage: 241.9+ KB


#### Tidiness 

df_image , df_info_json needs to be joined with archive df_twitter

##### Define. 
Create a master dataframe with all the dataframes combined

##### Code

In [548]:
twitter_dfs = pd.merge(df_twitter_clean, df_image_clean, how='inner', on=['tweet_id'])

In [549]:
twitter_dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1949 entries, 0 to 1948
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            1949 non-null   int64              
 1   timestamp           1949 non-null   datetime64[ns, UTC]
 2   source              1949 non-null   object             
 3   text                1949 non-null   object             
 4   expanded_urls       1949 non-null   object             
 5   rating_numerator    1949 non-null   int64              
 6   rating_denominator  1949 non-null   int64              
 7   name                1343 non-null   object             
 8   stage               302 non-null    object             
 9   jpg_url             1949 non-null   object             
 10  img_num             1949 non-null   int64              
 11  p1                  1949 non-null   object             
 12  p1_conf             1949 non-null 

In [550]:
twitter_dfs = pd.merge(twitter_dfs, df_info_json_clean, how='inner', on=['tweet_id'])

##### Test:

In [551]:
twitter_dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1942 entries, 0 to 1941
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            1942 non-null   int64              
 1   timestamp           1942 non-null   datetime64[ns, UTC]
 2   source              1942 non-null   object             
 3   text                1942 non-null   object             
 4   expanded_urls       1942 non-null   object             
 5   rating_numerator    1942 non-null   int64              
 6   rating_denominator  1942 non-null   int64              
 7   name                1336 non-null   object             
 8   stage               301 non-null    object             
 9   jpg_url             1942 non-null   object             
 10  img_num             1942 non-null   int64              
 11  p1                  1942 non-null   object             
 12  p1_conf             1942 non-null 